# Generate Folds for Nested CV on MIMIC dataset with Comfort Measure Patients Excluded

In [1]:
from sklearn.model_selection import StratifiedKFold
import mimic_pipeline.utils as utils
import pandas as pd

utils.seed_everything()
train = pd.read_csv("data/TRAIN-union-features-id-excluded-cmo.csv")
test = pd.read_csv("data/TEST-union-features-id-excluded-cmo.csv")
entire = pd.concat([train, test], axis=0)

X, y = entire.drop('hospital_expire_flag', axis=1), entire['hospital_expire_flag']
X.head()

,subject_id,hadm_id,icustay_id,preiculos,age,gcs_min,mechvent,urineoutput,heartrate_min,heartrate_max,...,pao2fio2_vent_min,admissiontype,pao2_max,pao2_min,paco2_max,paco2_min,ph_min,ph_max,aado2_min,aado2_max
0,88280,198866,254097,4777.0,43.0,15.0,0,3980.0,68.0,118.0,...,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,97666,158220,287837,1.0,50.0,15.0,0,1050.0,77.0,120.0,...,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6688,189787,231393,238.0,38.0,15.0,1,2000.0,101.0,116.0,...,NaN,0,458.0,139.0,52.0,34.0,7.21,7.47,284.0,284.0
3,90992,181692,207539,286.0,80.0,15.0,0,1145.0,46.0,76.0,...,NaN,0,168.0,80.0,41.0,27.0,7.36,7.42,NaN,NaN
4,29446,121959,278581,1.0,75.0,13.0,0,3040.0,67.0,92.0,...,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=utils.SEED)
for counter, (train_idx, test_idx) in enumerate(kf.split(X, y), 1):
    X_train, y_train, X_test, y_test = X.iloc[train_idx], y.iloc[train_idx], X.iloc[test_idx], y.iloc[test_idx]
    tmp_train = pd.concat([X_train, y_train], axis=1)
    tmp_test = pd.concat([X_test, y_test], axis=1)
    assert tmp_train.shape[0] + tmp_test.shape[0] == entire.shape[0]
    assert tmp_train.shape[1] == tmp_test.shape[1]
    print(f"{100*len(tmp_train[tmp_train['hospital_expire_flag'] == 1]) / len(tmp_train):.2f}%")
    print(f"{100*len(tmp_test[tmp_test['hospital_expire_flag'] == 1]) / len(tmp_test):.2f}%\n")
    tmp_train.to_csv(f"data/k-fold-no-cmo/TRAIN-union-features-fold{counter}.csv", index=False)
    tmp_test.to_csv(f"data/k-fold-no-cmo/TEST-union-features-fold{counter}.csv", index=False)

7.65%
7.64%

7.65%
7.64%

7.65%
7.64%

7.64%
7.65%

7.64%
7.65%

